import packages

In [ ]:
import numpy as np 
import os 
import sys
from pathlib import Path 
import pickle
from ase.io import read
from skimage.measure import regionprops

path_to_prn = os.path.abspath(
    '.../PoroNet')
sys.path.append(path_to_prn)
import poronet_functions as prn


Import the required MOFs and pore index

In [ ]:
#Here we chose the pores with diameter from 8.4 to 8.7 as examples
Required_MOFs=pickle.load(open(('Required_MOFs_84_87.pkl'), 'rb'))
Index_of_pbc_pores_84_87=pickle.load(open(('Index_of_pbc_pores_84_87.pkl'), 'rb'))

Calculte psd with loop structure

In [ ]:
csd_list=[]
error=[]
for i,MOF in enumerate(Required_MOFs):
  try:
    #set the path of the cif files
    cif_path = f".../Tobacco/{MOF}.cif"
    mat_atoms = read(cif_path)
    dgrid = prn.dgrid_from_atoms_cpu_no_aabb(mat_atoms, spacing=0.5)
    regions, maxima = prn.regions_from_dgrid_with_threshold_abs(dgrid, mask_thickness = 0, h=0.5, threshold_abs=1)
    maxima_radii = dgrid[tuple(maxima.T)]
    pbc_groups = prn.apply_pbc(regions, maxima, maxima_radii, mat_atoms, return_conn=True)
    props=regionprops(regions)
    shape=regions.shape
    pbc=Index_of_pbc_pores_84_87[i]
    bins_csd=np.linspace(0,15,31)
    y=prn.calculate_csd_with_pbc(props,pbc,pbc_groups,bins_csd,mat_atoms,shape)
    csd_list.append(y)
    print(f"{i+1}/{len(Required_MOFs)}")
  except Exception as e:
        error.append(i)
        continue    